# Notebook 4: Interactive Network Visualization with Bokeh

#### This notebook is based off of the following tutorial: https://melaniewalsh.github.io/Intro-Cultural-Analytics/06-Network-Analysis/02-Making-Network-Viz-with-Bokeh.html

In [1]:
# Import packages
import pandas as pd
import networkx
import matplotlib.pyplot as plt
import numpy as np
from bokeh.io import output_file, show, save
from bokeh.models import Range1d, Circle, ColumnDataSource, MultiLine
from bokeh.plotting import figure, from_networkx
from bokeh.palettes import Blues8, Reds8, Purples8, Oranges8, Viridis8, Spectral8
from bokeh.transform import linear_cmap

### Read edges csv (https://github.com/melaniewalsh/sample-social-network-datasets/tree/master/sample-datasets/game-of-thrones)

In [2]:
df = pd.read_csv('../data/got-edges.csv')
df

,Source,Target,Weight
0,Aemon,Grenn,5
1,Aemon,Samwell,31
2,Aerys,Jaime,18
3,Aerys,Robert,6
4,Aerys,Tyrion,5
...,...,...,...
347,Walder,Petyr,6
348,Walder,Roslin,6
349,Walton,Jaime,10
350,Ygritte,Qhorin,7


### Create a network (https://networkx.org/documentation/stable/reference/generated/networkx.convert_matrix.from_pandas_edgelist.html)

In [3]:
G =  (df, 'Source', 'Target', 'Weight')

# Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [("Character", "@index")]

# Create the plot
title = 'Game_of_Thrones_Network'
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

### Create a network graph object with spring layout (https://networkx.github.io/documentation/networkx-1.9/reference/generated/networkx.drawing.layout.spring_layout.html)

In [4]:
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node size and color
network_graph.node_renderer.glyph = Circle(radius=0.2, fill_color='skyblue')

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

#Add network graph to the plot
plot.renderers.append(network_graph)

In [5]:
# save plot
output_file(filename=f"{title}.html") 
save(plot, filename=f"{title}.html", title=title)
show(plot)

### Calculate the degree for each node and add it as a node attribute (https://networkx.org/documentation/stable/reference/generated/networkx.classes.function.set_node_attributes.html)

In [6]:
degrees = dict(networkx.degree(G))
networkx.set_node_attributes(G, name='degree', values=degrees)

In [7]:
# adjust degree so that the nodes with very small degrees are still visible
number_to_adjust_by = 0.1
adjusted_node_size = dict([(node, (degree/100)+number_to_adjust_by) for node, degree in networkx.degree(G)])
networkx.set_node_attributes(G, name='adjusted_node_size', values=adjusted_node_size)

In [8]:
#Choose attributes to size and color by
size_by_this_attribute = 'adjusted_node_size'
color_by_this_attribute = 'adjusted_node_size'

#Pick a color palette
color_palette = Blues8

# Establish which categories will appear when hovering over each node
HOVER_TOOLTIPS = [
       ("Character", "@index"),
        ("Degree", "@degree")
]

In [9]:
#Create the plot
title = 'GOT_Network_Size_And_Color'
plot = figure(tooltips = HOVER_TOOLTIPS,
              tools="pan,wheel_zoom,save,reset", active_scroll='wheel_zoom',
            x_range=Range1d(-10.1, 10.1), y_range=Range1d(-10.1, 10.1), title=title)

In [10]:
#Create the network graph object
network_graph = from_networkx(G, networkx.spring_layout, scale=10, center=(0, 0))

#Set node sizes and colors according to node degree (color as spectrum of color palette)
minimum_value_color = min(network_graph.node_renderer.data_source.data[color_by_this_attribute])
maximum_value_color = max(network_graph.node_renderer.data_source.data[color_by_this_attribute])
network_graph.node_renderer.glyph = Circle(radius=size_by_this_attribute, fill_color=linear_cmap(color_by_this_attribute, color_palette, minimum_value_color, maximum_value_color))

#Set edge opacity and width
network_graph.edge_renderer.glyph = MultiLine(line_alpha=0.5, line_width=1)

plot.renderers.append(network_graph)
output_file(filename=f"{title}.html") 
save(plot, filename=f"{title}.html", title=title)

'/home/jupyter-jpclemente97/dsd2025/final_code/GOT_Network_Size_And_Color.html'